In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from joblib import load
from tqdm import tqdm
import warnings
 
warnings.filterwarnings("ignore")
import glob
import os
from collections import OrderedDict
from src.layers.classifiers import TokenClassifier, MixedClassifier

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import cm

In [2]:
dict = {
    "AGN": 0,
    "QSO": 1,
    "EA": 2,
    "YSO": 3,
    "SNIa": 4,
    "CV/Nova": 5,
    "RRLc": 6,
    "RSCVn": 7,
    "Blazar": 8,
    "SNII": 9,
    "EB/EW": 10,
    "LPV": 11,
    "CEP": 12,
    "RRLab": 13,
    "Periodic-Other": 14,
    "DSCT": 15,
    "SNIbc": 16,
    "SLSN": 17,
    "TDE": 18,
    "SNIIb": 19,
    "SNIIn": 20,
    "Microlensing": 21
}

In [3]:

path = '/home/mdelafuente/'#sys.argv[2]

device = torch.device(f"cuda:2" if torch.cuda.is_available() else "cpu")


abs_path = '//home/mdelafuente/pipeline/pipeline/training/lc_classifier_ztf/ATAT_ALeRCE/results/ZTF_ff/LC/cosine-four-trans-v0/'
path_args = glob.glob(f"{abs_path}*args*")[0]
print(path_args)
import yaml
with open(path_args, 'r') as file:
    args = yaml.safe_load(file)

//home/mdelafuente/pipeline/pipeline/training/lc_classifier_ztf/ATAT_ALeRCE/results/ZTF_ff/LC/cosine-four-trans-v0/args.yaml


In [4]:
print(args)

{'lc': {'input_size': 1, 'embedding_size': 192, 'embedding_size_sub': 384, 'num_heads': 4, 'num_encoders': 3, 'Tmax': 1500.0, 'num_harmonics': 4, 'pe_type': 'tm', 'encoder_type': 'Linear', 'max_pool_kernel': 5, 'cnn_kernel': 5, 'num_bands': 2}, 'ft': {'embedding_size': 128, 'embedding_size_sub': 256, 'num_heads': 4, 'num_encoders': 3, 'encoder_type': 'Linear', 'length_size': 0, 'list_time_to_eval': None}, 'general': {'experiment_type': 'lc', 'experiment_name': 'cosine-four-trans-v0', 'name_dataset': 'ztf_ff', 'data_root': '/home/mdelafuente/pipeline/pipeline/training/lc_classifier_ztf/ATAT_ALeRCE/data/datasets/h5file/', 'use_lightcurves': True, 'use_lightcurves_err': False, 'use_metadata': False, 'use_features': False, 'use_sampler': 0, 'batch_size': 256, 'num_epochs': 100, 'patience': 10, 'lr': 0.001, 'use_cosine_decay': False, 'use_gradient_clipping': False, 'use_mask_detection': False, 'use_time_nondetection': False, 'force_online_opt': False, 'online_opt_tt': False, 'use_QT': False

In [5]:
print('Initializing models') 
from src.layers.cATAT import LightCurveClassifier, LightCurveTransformer
model = LightCurveTransformer(**args['lc']) 


print('Loading checkpoints')
checkpoint_path_clip = glob.glob(
    f"{abs_path}*my_best_checkpoint*"
) 
print(checkpoint_path_clip[-1])
checkpoint_clip = torch.load(checkpoint_path_clip[-1], map_location=torch.device('cuda:2'))
od_atat = OrderedDict() 

for key in checkpoint_clip["state_dict"].keys(): 
        #print(key)
        
    if 'project' in key:
        continue
    else:
        od_atat[key.replace("model.transformer.", "")] = checkpoint_clip["state_dict"][key]

model.load_state_dict(od_atat,strict=True)

Initializing models
Loading checkpoints
//home/mdelafuente/pipeline/pipeline/training/lc_classifier_ztf/ATAT_ALeRCE/results/ZTF_ff/LC/cosine-four-trans-v0/my_best_checkpoint-step=3529.ckpt


<All keys matched successfully>

In [6]:
seed = 0
from src.data.modules.LitData  import LitData
args['general']['data_root'] = 'data/datasets/ZTF_ff/final/LC_MD_FEAT_240627_windows_200_12'
args['general']['data_root'] = 'data/datasets/ZTF_ff/final/LC_MD_FEAT_240627_windows_200_12'
args['general']['use_sampler'] = False
pl_datal = LitData(**args['general'])
dataloader = pl_datal.val_dataloader() 
    

KWARGS
{'experiment_type': 'lc', 'experiment_name': 'cosine-four-trans-v0', 'name_dataset': 'ztf_ff', 'use_lightcurves': True, 'use_lightcurves_err': False, 'use_metadata': False, 'use_features': False, 'use_sampler': False, 'num_epochs': 100, 'patience': 10, 'lr': 0.001, 'use_cosine_decay': False, 'use_gradient_clipping': False, 'use_mask_detection': False, 'use_time_nondetection': False, 'force_online_opt': False, 'online_opt_tt': False, 'use_QT': False, 'load_pretrained_model': False, 'src_checkpoint': '.', 'use_augmented_dataset': False, 'change_clf': False, 'num_classes': 22}
using set validation total of idx : 40698,                 use_lightcurves True, use_metadata False, use_features False,                     use MTA False
list_time_to_eval:  [16, 32, 64, 128, 256, 512, 1024, 2048]
NOT USING SAMPLER


In [11]:
target = None
preds_out = None

model.eval().to(device = 'cuda:2')  

for b1 in tqdm(dataloader):

    #print(b1.keys())
    b1 = {key:value.to(device = 'cuda:2') for key, value in b1.items()}
    t = b1['labels']
    del b1['labels']
    lc_emb = model(**b1)  
    
    

    preds_out = (
        np.concatenate([preds_out, torch.argmax(lc_emb, axis=1).cpu().numpy()])
        if preds_out is not None
        else torch.argmax(lc_emb, axis=1).cpu().detach().numpy()
    )

    target = (
        np.concatenate([target, t.cpu().detach().numpy()])
        if target is not None
        else t.cpu().numpy()
    )

100%|██████████| 159/159 [00:22<00:00,  7.11it/s]


In [35]:
np.unique(target).shape,np.unique(preds_out).shape

((22,), (133,))

In [ ]:
import matplotlib.pyplot as plt
from utils.plottinglib import elasticc_confusion_matrix

out_metrics_balto = classification_report(
    target, results, target_names=list(dict.keys()), output_dict=True
)["macro avg"]

template_balto = ""

for key in out_metrics_balto.keys():
    template_balto += " {} : {:.3f} ".format(key.upper(), out_metrics_balto[key])
fig, axes = plt.subplots(1, 1, figsize=(12, 12))
elasticc_confusion_matrix(
    y_true=target,
    y_pred=results,
    classes= np.array(list(dict.keys())),
    ax=axes,
    normalize=True,
    title=f" FClassifier Results [TEST] \n\n {template_balto}",
)